In [ ]:
import logging
from kafka import KafkaConsumer
import requests
import json

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Replace with your Kafka broker address(es)
brokers = ["ec2-54-80-241-235.compute-1.amazonaws.com:9092"]

# Replace with your topic name
topic = "three"
# KServe inference service URL
kserve_url = 'http://dvc.kubeflow-user-example-com.svc.cluster.local/v1/models/dvc:predict'

# Create a Kafka consumer
logger.info("Creating Kafka consumer...")
consumer = KafkaConsumer(
    topic,
    bootstrap_servers=brokers,
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='my-group',
    value_deserializer=lambda v: v.decode('utf-8')
)

logger.info(f"Subscribed to topic: {topic}")


# Function to send data to KServe
def send_to_kserve(data):
    headers = {'Content-Type': 'application/json'}
    response = requests.post(kserve_url, headers=headers, data=json.dumps(data))
    return response.json()

try:
    for message in consumer:
        try:
            message = json.loads(message.value)  # Message value is already a dict
            print(f"Received message: {message}")

            # Prepare data for KServe
            kserve_payload = {
                "instances": [list(message.values())]  # Reshape the data to 2D array
            }

            # Send data to KServe
            kserve_response = send_to_kserve(kserve_payload)
            
            print(f"KServe response: {kserve_response}")
        except json.JSONDecodeError:
            print(f"Received non-JSON message: {message.value}")
        except Exception as e:
                logger.error(f"Error sending to KServe: {e}")
        #logger.info(f"Partition: {message.partition}, Offset: {message.offset}")
except KeyboardInterrupt:
    logger.info("Consumer stopped manually.")
finally:
    consumer.close()
    logger.info("Consumer closed.")
